### Downloading Imageio library in local environment

In [0]:
!pip install imageio

### Downloading DICOM Dataset from Github

In [0]:
!git clone https://github.com/Heisenberg0203/PUBG.git

In [0]:
!ls PUBG/MRi/SCD2001_010

###Importing Required Libraries

In [0]:
import imageio
import matplotlib.pyplot as plt
import numpy as np

1. Reading Image

In [0]:

image = imageio.imread('PUBG/MRi/SCD2001_010/SCD2001_MR_001.dcm')

print('Image type:', type(image))
print('Shape of image array:', image.shape)

2. Metadata of Dicom Image

In [0]:
image.meta.keys()

3. Plotting Image

In [0]:
plt.imshow(image,)
plt.show()

3.2 Plotting Image in greyscale

In [0]:
plt.imshow(image,cmap='gray')
plt.show()

4. Plotting Image : - Adjusting Contrast to view better

In [0]:
#vmin and vmax control the color contrast between values. 
plt.imshow(image, cmap='gray', vmin=-50, vmax=50)
plt.axis('off')
plt.show()

### N -Dimensional Images

In [0]:
##n -dim images

image1 = imageio.imread('PUBG/MRi/SCD2001_010/SCD2001_MR_001.dcm')
image2 = imageio.imread('PUBG/MRi/SCD2001_010/SCD2001_MR_002.dcm')
image3 = imageio.imread('PUBG/MRi/SCD2001_010/SCD2001_MR_003.dcm')

--Stacking the 3Images to get Volumetric Data

In [0]:
volume = np.stack([image1, image2, image3], axis=0)
print('Volume dimensions:', volume.shape)

5. Reading Complete Directory

In [0]:

volume = imageio.volread('PUBG/MRi/SCD2001_010/')

print('Available metadata:', volume.meta.keys())
print('Shape of image array:', volume.shape)

The amount of physical space covered by an image is its field of view, which is calculated from two properties:

Array shape, the number of data elements on each axis. Can be accessed with the shape attribute.
Sampling resolution, the amount of physical space covered by each pixel. Sometimes available in metadata (e.g., meta['sampling']).

In [0]:

n0, n1, n2 = volume.shape
d0, d1, d2 = volume.meta['sampling']
n0 * d0, n1 * d1, n2 * d2

### Plotting Multiple Images at Once

In [0]:
# Initialize figure and axes grid
fig, axes = plt.subplots(nrows=2, ncols=1)

# Draw an image on each subplot
axes[0].imshow(image1, cmap='gray')
axes[1].imshow(image2, cmap='gray')

# Remove ticks/labels and render
axes[0].axis('off')
axes[1].axis('off')
plt.show()

##Plotting several slices of Dataset

In [0]:
fig, axes = plt.subplots(nrows=1, ncols=4)

# Loop through subplots and draw image
for ii in range(4):
    im = volume[ii * 5, :, :]
    axes[ii].imshow(image, cmap='gray')
    axes[ii].axis('off')

# Render the figure
plt.show()

###Determining Aspect Ratios to get Different View of Image

To calculate aspect ratios, first retrieve the sampling rates along each dimension: d0, d1, d2 = vol.meta['sampling']. To plot "axis 0 vs. axis 2", you must determine how much to widen pixels on "axis 0": d0 / d2

In [0]:

image1 = vol[:, 128, :]
image2 = vol[:, :, 128]

# Compute aspect ratios
d0, d1, d2 = volume.meta['sampling']
asp1 = d0 / d2
asp2 = d0 / d1

# Plot the images on a subplots array 
fig, axes = plt.subplots(nrows=2, ncols=1)
axes[0].imshow(image1, cmap='gray',aspect=16/3 )
axes[1].imshow(image2, cmap='gray', aspect = 4)
plt.show()

### Plotting GrayScale Image

In [0]:
image = imageio.imread('PUBG/MRi/SCD2001_010/SCD2001_MR_005.dcm')
print('Data type:', image.dtype)
print('Min. value:', image.min())
print('Max value:', image.max())

# Plot the grayscale image
plt.imshow(image, vmin=0, vmax=255)
plt.colorbar()
plt.show()

Histograms display the distribution of values in your image by binning each element by its intensity then measuring the size of each bin.

The area under a histogram is called the cumulative distribution function. It measures the frequency with which a given range of pixel intensities occurs.

## Intensity Measurement 

In [0]:
import scipy.ndimage as ndi

# Generate histogram
hist = ndi.histogram(image, min=0, max=255, bins=256)

# Calculate cumulative distribution
cdf = hist.cumsum() / hist.sum()

# Plot the histogram and CDF
fig, axes = plt.subplots(2,1, sharex=True)
axes[0].plot(hist, label='Histogram')
axes[1].plot(cdf, label='CDF')

The full image is a 3D time series spanning a single heartbeat. These data are used by radiologists to measure the ejection fraction: the proportion of blood ejected from the left ventricle during each stroke.



## Creating a Mask of an Image('Heart')

In [0]:
image_filter = ndi.median_filter(image, size=3)

# Select high-intensity pixels
mask_start = np.where(image_filter > 60, 1, 0)
mask = ndi.binary_closing(mask_start)

# Label the objects in "mask"
labels, nlabels = ndi.label(mask)
print('Num. Labels:', nlabels)

###Plotting the Overlay Mask

In [0]:
# Label the image "mask"
labels, nlabels = ndi.label(mask)

# Select left ventricle pixels
lv_val = labels[128, 128]
lv_mask = np.where(labels == lv_val, 1, np.nan)

# Overlay selected label
plt.imshow(lv_mask, cmap='rainbow')
plt.show()

##Plotting only a specific part - left ventricle

In [0]:
# Create left ventricle mask
labels, nlabels = ndi.label(mask)
lv_val = labels[128, 128]
lv_mask = np.where(labels == lv_val, 1, 0)

In [0]:

# Find bounding box of left ventricle
bboxes = ndi.find_objects(lv_mask)
print('Number of objects:', len(bboxes))
print('Indices for first box:', bboxes[0])

In [0]:
# Crop to the left ventricle (index 0)
im_lv = im[bboxes[0]]

# Plot the cropped image
plt.imshow(im_lv)

Quantifying tissue morphology, or shape is one primary objective of biomedical imaging. The size, shape, and uniformity of a tissue can reveal essential health insights.

##Finding Volume of Image

In [0]:
pixels =ndi.sum(1, labels, index=1)
unitvolume = vol.meta['sampling']

In [0]:
volume.shape

In [0]:
volume_of_left_ventricle = pixels *unitvolume[0]*unitvolume[1]*unitvolume[2]

In [0]:
volume_of_left_ventricle